## Linear Regression

O objetivo é mostrar de forma simples o passo a passo na criação de um modelo:
1. Problema do Negócio
2. Captura dos dados
3. Tratamento e Limpeza dos dados
4. Criação do modelo
5. Validação do modelo
6. Refactor
7. Salvando um modelo e utilizando

## Imports
- pandas -> manipulação de dados no formato de tabelas
- sklearn -> algorítimos de machine learning
- numpy -> biblioteca para cálculos
- pickle -> gravar binários(nossos modelos)

In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
import numpy as np
import pickle

## 1. Problema

Temos dados referente a um bairro, e queremos prever:
- **PREÇO DAS CASAS**

E temos diversas variáveis que podem alterar o preço dela....

Alguns dados importantes do nosso "dataset":
- sqft_living, the total house square footage of the house
- sqft_basement, size of the basement
- sqft_above = sqft_living - sqft_basement
- sqft_lot, lot size of the house
- sqft_living15, the average house square footage of the 15 closest houses
- sqft_lot15, the average lot square footage of the 15 closest houses

## 2. Captura dos Dados
Vamos pegar o dados de um dataset do kaggle para estudo:
- link dataset: https://www.kaggle.com/harlfoxem/housesalesprediction

### Trazendo para o Jupyternotebook os dados

In [2]:
dataset_raw = pd.read_csv('kc_house_data.csv')

### Visualizando estes dados

In [4]:
dataset_raw.head(10)

,id,date,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,waterfront,view,...,grade,sqft_above,sqft_basement,yr_built,yr_renovated,zipcode,lat,long,sqft_living15,sqft_lot15
0,7129300520,20141013T000000,221900.0,3,1.00,1180,5650,1.0,0,0,...,7,1180,0,1955,0,98178,47.5112,-122.257,1340,5650
1,6414100192,20141209T000000,538000.0,3,2.25,2570,7242,2.0,0,0,...,7,2170,400,1951,1991,98125,47.7210,-122.319,1690,7639
2,5631500400,20150225T000000,180000.0,2,1.00,770,10000,1.0,0,0,...,6,770,0,1933,0,98028,47.7379,-122.233,2720,8062
3,2487200875,20141209T000000,604000.0,4,3.00,1960,5000,1.0,0,0,...,7,1050,910,1965,0,98136,47.5208,-122.393,1360,5000
4,1954400510,20150218T000000,510000.0,3,2.00,1680,8080,1.0,0,0,...,8,1680,0,1987,0,98074,47.6168,-122.045,1800,7503
5,7237550310,20140512T000000,1225000.0,4,4.50,5420,101930,1.0,0,0,...,11,3890,1530,2001,0,98053,47.6561,-122.005,4760,101930
6,1321400060,20140627T000000,257500.0,3,2.25,1715,6819,2.0,0,0,...,7,1715,0,1995,0,98003,47.3097,-122.327,2238,6819
7,2008000270,20150115T000000,291850.0,3,1.50,1060,9711,1.0,0,0,...,7,1060,0,1963,0,98198,47.4095,-122.315,1650,9711
8,2414600126,20150415T000000,229500.0,3,1.00,1780,7470,1.0,0,0,...,7,1050,730,1960,0,98146,47.5123,-122.337,1780,8113
9,3793500160,20150312T000000,323000.0,3,2.50,1890,6560,2.0,0,0,...,7,1890,0,2003,0,98038,47.3684,-122.031,2390,7570


In [5]:
dataset_raw.columns

Index(['id', 'date', 'price', 'bedrooms', 'bathrooms', 'sqft_living',
       'sqft_lot', 'floors', 'waterfront', 'view', 'condition', 'grade',
       'sqft_above', 'sqft_basement', 'yr_built', 'yr_renovated', 'zipcode',
       'lat', 'long', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

In [6]:
# vaçodamdp 
dataset_raw.shape

(21613, 21)

## 3. Tratamento e Limpeza dos dados
Conforme a função acima, nem todos os dados fazem sentido para nosso modelo, portanto será necessário retirar algumas colunas

As colunas escolhidas para serem retiradas:
- id
- date
- zipcode
- lat
- long
- view
- waterfront

### Deletando as colunas que não iremos utilziar

In [7]:
dataset_raw.drop('id',inplace = True,axis = 1)
dataset_raw.drop('date',inplace = True,axis = 1)
dataset_raw.drop('zipcode',inplace = True,axis = 1)
dataset_raw.drop('lat',inplace = True,axis = 1)
dataset_raw.drop('long',inplace = True,axis = 1)
dataset_raw.drop('view',inplace = True,axis = 1)
dataset_raw.drop('waterfront',inplace = True,axis = 1)

### Validando que as colunas foram retiradas

In [8]:
dataset_raw.columns

Index(['price', 'bedrooms', 'bathrooms', 'sqft_living', 'sqft_lot', 'floors',
       'condition', 'grade', 'sqft_above', 'sqft_basement', 'yr_built',
       'yr_renovated', 'sqft_living15', 'sqft_lot15'],
      dtype='object')

## 4. Criação do modelo

### Instanciando um modelo

In [13]:
modelo_inicial = LinearRegression()

### Escolhendo quais variáveis serão importantes para o modelo
- Y - O que desejo prever - PREÇO
- X - Variáveis que podem fazer meu preço variar - No caso foi escolhida todas como exemplo

In [9]:
y = dataset_raw['price']
x = dataset_raw.iloc[:,1:]

### Dividindo os dados entre "treino" e "teste"

In [10]:
x_treino, x_teste, y_treino, y_teste = train_test_split(x, 
                                                        y, 
                                                        test_size = 0.3, # 30% de teste
                                                        random_state = 10 #random state setta a arbitrariedade
                                                       )

### Validando que foi dividido o dataset

In [11]:
imprimir_dados = f"""
                Minha quantidade de dados iniciais: {dataset_raw.shape[0]}
                Minha quantidade de dados x_treino: {x_treino.shape[0]}
                Minha quantidade de dados x_teste: {x_teste.shape[0]}
                Minha quantidade de dados y_treino: {y_treino.shape[0]}
                Minha quantidade de dados y_teste: {y_teste.shape[0]}  
                
                {dataset_raw.shape[0]} x 30% = {dataset_raw.shape[0]*.3}
                
                
                """

print(imprimir_dados)


                Minha quantidade de dados iniciais: 21613
                Minha quantidade de dados x_treino: 15129
                Minha quantidade de dados x_teste: 6484
                Minha quantidade de dados y_treino: 15129
                Minha quantidade de dados y_teste: 6484  
                
                21613 x 30% = 6483.9
                
                
                


### Treinando o Modelo

In [14]:
modelo_inicial.fit(x_treino, y_treino)

LinearRegression()

### Usando o modelo

In [19]:
dataset_raw.head(7)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,3,7,1180,0,1955,0,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,3,7,2170,400,1951,1991,1690,7639
2,180000.0,2,1.00,770,10000,1.0,3,6,770,0,1933,0,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,5,7,1050,910,1965,0,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,3,8,1680,0,1987,0,1800,7503
5,1225000.0,4,4.50,5420,101930,1.0,3,11,3890,1530,2001,0,4760,101930
6,257500.0,3,2.25,1715,6819,2.0,3,7,1715,0,1995,0,2238,6819


In [20]:
# Pegando a primeira
linha = 6
teste = x.iloc[linha]
resultado_esperado = y.iloc[linha]
print(f'Peguei os valores da linha {linha} preço esperado: {resultado_esperado}')

Peguei os valores da linha 6 preço esperado: 257500.0


In [21]:
resultado_modelo = modelo_inicial.predict([teste])[0]
imprime_resultado_modelo = f"""
                                Resultado esperado: {resultado_esperado}
                                Resultado do meu modelo: {resultado_modelo}
                                Portanto eu errei: {resultado_esperado-resultado_modelo}
"""
print(imprime_resultado_modelo)


                                Resultado esperado: 257500.0
                                Resultado do meu modelo: 341567.689211417
                                Portanto eu errei: -84067.68921141699



Portanto esse modelo é ruim, pois erra demais.

Porém é interessante visualizar de uma forma geral

## 5.Validação do modelo

Utilizando a função score, ele pega os dados de teste(não o de treino) e valida se o modelo está acertando

Dessa forma, podemos ver todas as validações de acerto/erro

In [17]:
modelo_inicial.score(x_teste, y_teste)

0.6232905949591507

## 7. Refactor

Como o modelo tem um % de acerto de aproximadamente 63%, é necessário "ajusta-lo".

A forma mais simples, é trazendo menos complexidade ao modelo, com menos variáveis.

### Alterando nossos inputs no modelo
- somente a variável sqft_lot é importante, portanto:
- y -> preço
- x -> sqft_lot(no modelo anterior foram todas as outras variáveis)

In [22]:
y2 = dataset_raw['price']
x2 = dataset_raw['sqft_lot']

### Dividindo os dados

In [23]:
x_treino2, x_teste2, y_treino2, y_teste2 = train_test_split(x2, 
                                                        y2, 
                                                        test_size = 0.3, 
                                                        random_state = 10 #random state setta a arbitrariedade
                                                       )

### Criando um novo modelo

In [24]:
modelo_segundo = LinearRegression()

### Ajustando dados

Como é somente 1 X (feature ou variável independente), é necessário ajustar para que o modelo aceite

No caso, estamos pegando uma series(tipo de formato do python) e alterando o "tamanho" da matriz para 2 dimensões

In [25]:
x_treino2 = x_treino2.values.reshape(x_treino2.shape[0],1)
x_teste2 = x_teste2.values.reshape(x_teste2.shape[0],1)

### Treinando o Modelo

In [26]:
modelo_segundo.fit(x_treino2, y_treino2)

LinearRegression()

### Utilizando o modelo

In [27]:
dataset_raw.head(7)

,price,bedrooms,bathrooms,sqft_living,sqft_lot,floors,condition,grade,sqft_above,sqft_basement,yr_built,yr_renovated,sqft_living15,sqft_lot15
0,221900.0,3,1.00,1180,5650,1.0,3,7,1180,0,1955,0,1340,5650
1,538000.0,3,2.25,2570,7242,2.0,3,7,2170,400,1951,1991,1690,7639
2,180000.0,2,1.00,770,10000,1.0,3,6,770,0,1933,0,2720,8062
3,604000.0,4,3.00,1960,5000,1.0,5,7,1050,910,1965,0,1360,5000
4,510000.0,3,2.00,1680,8080,1.0,3,8,1680,0,1987,0,1800,7503
5,1225000.0,4,4.50,5420,101930,1.0,3,11,3890,1530,2001,0,4760,101930
6,257500.0,3,2.25,1715,6819,2.0,3,7,1715,0,1995,0,2238,6819


In [31]:
linha = 2
teste02 = x2.iloc[linha]
teste02 = teste02.reshape(1,1)
resultado_esperado02 = y2.iloc[linha]
print(f'Peguei os valores da linha {linha} preço esperado: {resultado_esperado02}')

Peguei os valores da linha 2 preço esperado: 180000.0


In [32]:
resultado_modelo02 = modelo_segundo.predict(teste02)[0]
imprime_resultado_modelo02 = f"""
                                Resultado esperado: {resultado_esperado02}
                                Resultado do meu modelo: {resultado_modelo02}
                                Portanto eu errei: {resultado_esperado02-resultado_modelo02}
"""
print(imprime_resultado_modelo02)


                                Resultado esperado: 180000.0
                                Resultado do meu modelo: 536838.2778025
                                Portanto eu errei: -356838.27780249994



### Validando o Modelo

In [30]:
modelo_segundo.score(x_teste2, y_teste2)

0.007194397845567635

Perceba que mesmo que ele tenha conseguido chegar perto no exemplo acima, seu acerto é muito baixo

## 6. Utilizando o Modelo

Para que não seja preciso, ficar fazendo todo o processo acima, é interessante gravarmos um modelo em um binário

### Salvando um modelo

In [46]:
pickle.dump(modelo_segundo, open('teste.sav','wb'))

### Carregando um modelo

In [48]:
modelo_carregado = pickle.load(open('teste.sav','rb'))

### Utilizando o modelo carregado

In [49]:
modelo_carregado.predict(teste02)[0]

534540.7412365694

In [51]:
resultado_modelo02

534540.7412365694

## Fim